In [ ]:
# Metrics and plots from Hands-on Machine Learning with Scikit-learn, Keras and Tensorflow

%matplotlib inline

import pandas as pd
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import (confusion_matrix, precision_score, recall_score, f1_score, roc_auc_score,
                             precision_recall_curve, roc_curve)
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

from model import _column_transformer

df = pd.read_csv('train.csv')
X = df.drop(columns=['Survived', 'PassengerId', 'Name', 'Ticket', 'Cabin'])
y = df['Survived']
column_transformer = _column_transformer(X.columns)

ada_boost = AdaBoostClassifier()
gradient_boosting = GradientBoostingClassifier()
random_forest = RandomForestClassifier()
k_neighbors = KNeighborsClassifier()
svc = SVC(kernel='poly')
classifiers = [ada_boost, gradient_boosting, random_forest, k_neighbors, svc]

def preds_and_scores(clf):
    pipeline = make_pipeline(column_transformer, clf)
    y_pred = cross_val_predict(pipeline, X, y, cv=3)
    if hasattr(clf, 'decision_function'):
        y_scores = cross_val_predict(pipeline, X, y, cv=3, method='decision_function')
    else:
        y_scores = cross_val_predict(pipeline, X, y, cv=3, method='predict_proba')
        y_scores = y_scores[:, 1]  # Get only positive cases
    return y_pred, y_scores

def metrics(clf):
    y_pred, y_scores = preds_and_scores(clf)
    return {'Confusion matrix': confusion_matrix(y, y_pred).tolist(),
            'Precision': precision_score(y, y_pred),
            'Recall': recall_score(y, y_pred),
            'F1 score': f1_score(y, y_pred),
            'ROC AUC score': roc_auc_score(y, y_scores)}

def plot_precision_recall(clf):
    _, y_scores = preds_and_scores(clf)
    precisions, recalls, thresholds = precision_recall_curve(y, y_scores)
    plt.plot(thresholds, precisions[:-1], 'b--', label='Precision')
    plt.plot(thresholds, recalls[:-1], 'g-', label='Recall')
    plt.legend()
    plt.gca().set_xlabel('Thresholds')  # gca: get current axis
    
def plot_roc_curves():
    for clf in classifiers:
        _, y_scores = preds_and_scores(clf)
        fpr, tpr, thresholds = roc_curve(y, y_scores)
        plt.plot(fpr, tpr, linewidth=2, label=type(clf).__name__)
    plt.plot([0, 1], [0, 1], 'k--')  # Diagonal
    plt.legend()
    plt.gca().set_xlabel('False Positive Rate')
    plt.gca().set_ylabel('True Positive Rate (Recall)')

In [ ]:
metrics(ada_boost)

In [ ]:
metrics(gradient_boosting)

In [ ]:
metrics(random_forest)

In [ ]:
metrics(k_neighbors)

In [ ]:
metrics(svc)

`SVC` is good at finding true negatives but very bad true positives - it is just predicting more negatives than the others. `GradientBoostingClassifier` gets the most true negatives (deaths) while `AdaBoostClassifier` gets the most true positives (survivals). `GradientBoostingClassifier` has the better precision while `AdaBoostClassifier` has the best recall and their F1 score is basically the same. `GradientBoostingClassifier` has the best ROC AUC.

In [ ]:
plot_precision_recall(ada_boost)

In [ ]:
plot_precision_recall(gradient_boosting)

In [ ]:
plot_precision_recall(random_forest)

In [ ]:
plot_precision_recall(k_neighbors)

In [ ]:
plot_precision_recall(svc)

In [ ]:
plot_roc_curves()